<a href="https://colab.research.google.com/github/DajanaSanchez/BirdMigrationClimateAnalysis/blob/main/CHELSA_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#defining drive and list files already downloaded
import os

# Define the folder path
drive_folder = "/content/drive/My Drive/CHELSA_Data"
existing_files = set(os.listdir(drive_folder))  # Store already downloaded files

print(f"Found {len(existing_files)} existing files.")

Found 4233 existing files.


In [ ]:
#loading the file list so script only downloads missing files
file_list_path = "/content/drive/My Drive/envidatS3paths.txt"

with open(file_list_path, "r") as f:
    all_urls = [url.strip() for url in f.readlines()]  # Read file URLs

In [ ]:
# Extract filenames from URLs to filter out already downloaded files
urls_to_download = [url for url in all_urls if os.path.basename(url) not in existing_files]

print(f"{len(urls_to_download)} files still need to be downloaded.")

0 files still need to be downloaded.


In [ ]:
#downloading files
for url in urls_to_download:
    filename = os.path.basename(url)
    destination = os.path.join(drive_folder, filename)

    print(f"Downloading: {filename}")
    !wget -c "{url}" -O "{destination}"

Downloading: CHELSA_bio5_2071-2100_ukesm1-0-ll_ssp585_V.2.1.tif
--2025-03-04 21:47:29--  https://os.zhdk.cloud.switch.ch/chelsav2/GLOBAL/climatologies/2071-2100/UKESM1-0-LL/ssp585/bio/CHELSA_bio5_2071-2100_ukesm1-0-ll_ssp585_V.2.1.tif
Resolving os.zhdk.cloud.switch.ch (os.zhdk.cloud.switch.ch)... 86.119.32.16, 2001:620:5ca1:101::1:125
Connecting to os.zhdk.cloud.switch.ch (os.zhdk.cloud.switch.ch)|86.119.32.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 136768983 (130M) [image/tiff]
Saving to: ‘/content/drive/My Drive/CHELSA_Data/CHELSA_bio5_2071-2100_ukesm1-0-ll_ssp585_V.2.1.tif’

/content/drive/My D 100%[===================>] 130.43M  17.7MB/s    in 8.5s    

2025-03-04 21:47:38 (15.3 MB/s) - ‘/content/drive/My Drive/CHELSA_Data/CHELSA_bio5_2071-2100_ukesm1-0-ll_ssp585_V.2.1.tif’ saved [136768983/136768983]

Downloading: CHELSA_bio6_2071-2100_ukesm1-0-ll_ssp585_V.2.1.tif
--2025-03-04 21:47:39--  https://os.zhdk.cloud.switch.ch/chelsav2/GLOBAL/climatologi

In [ ]:
!pip install rasterio geopandas pandas pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 69.0 MB/s eta 0:00:00


In [ ]:
import os
import glob

# Set the path to your CHELSA data folder
data_folder = "/content/drive/My Drive/CHELSA_Data/"

tif_files = glob.glob(os.path.join(data_folder, "*.tif"))

# Filter to include only GFDL-ESM4 model
tif_files = [f for f in tif_files if "gfdl-esm4" in f]
print(f"Using only GFDL-ESM4 model: {len(tif_files)} files selected.")

print("Example filenames:")
for file in tif_files[:5]:  # Print only a few examples
    print(os.path.basename(file))

Using only GFDL-ESM4 model: 846 files selected.
Example filenames:
CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_01_2011_2040_norm.tif
CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_02_2011_2040_norm.tif
CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_03_2011_2040_norm.tif
CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_04_2011_2040_norm.tif
CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_05_2011_2040_norm.tif


In [ ]:
import re
import os

def parse_filename(filename):
    filename = os.path.basename(filename)  # Extract only the filename

    # ✅ Standard format (with month) - Using your working regex
    pattern1 = re.match(
        r"CHELSA_([\w-]+)_([\w-]+)_([\w-]+)_(ssp\d{3})_([\w]+)_(\d{2})_(\d{4})_(\d{4})_norm\.tif",
        filename
    )

    # ✅ Alternative format (no month)
    pattern2 = re.match(
        r"CHELSA_([\w-]+)_([\d-]+)_([\w-]+)_(ssp\d{3})_V\.\d+\.\d+\.tif",
        filename
    )

    # ✅ If first pattern matches (with month)
    if pattern1:
        model, ensemble, bias_correction, ssp, variable, month, start_year, end_year = pattern1.groups()
        time_period = f"{start_year}-{end_year}"

    # ✅ If second pattern matches (no month)
    elif pattern2:
        variable, time_period, model, ssp = pattern2.groups()
        ensemble = bias_correction = month = None  # These don't exist in this format

    else:
        print(f"⚠️ Filename did not match any pattern: {filename}")
        return None, None, None, None, None, None, None  # Return None if filename doesn't match

    print(f"✅ Filename: {filename}")
    print(f"Model: {model}, Ensemble: {ensemble}, Bias Correction: {bias_correction}, SSP: {ssp}, Variable: {variable}, Month: {month}, Time Period: {time_period}")
    print("-" * 50)

    return model, ensemble, bias_correction, ssp, variable, month, time_period



# Example test
test_filenames = [
    "CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_01_2011_2040_norm.tif",
    "CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_02_2011_2040_norm.tif",
    "CHELSA_gdgfgd0_2071-2100_gfdl-esm4_ssp126_V.2.1.tif",
    "CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_03_2011_2040_norm.tif",
    "CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_04_2011_2040_norm.tif",
    "CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_05_2011_2040_norm.tif"
]

# Test the function on sample filenames
for file in test_filenames:
    parse_filename(file)

✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_01_2011_2040_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp585, Variable: tasmin, Month: 01, Time Period: 2011-2040
--------------------------------------------------
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_02_2011_2040_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp585, Variable: tasmin, Month: 02, Time Period: 2011-2040
--------------------------------------------------
✅ Filename: CHELSA_gdgfgd0_2071-2100_gfdl-esm4_ssp126_V.2.1.tif
Model: gfdl-esm4, Ensemble: None, Bias Correction: None, SSP: ssp126, Variable: gdgfgd0, Month: None, Time Period: 2071-2100
--------------------------------------------------
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_03_2011_2040_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp585, Variable: tasmin, Month: 03, Time Period: 2011-2040
-------------------------------------------

In [ ]:
import os

output_file = "/content/drive/My Drive/CHELSA_Data/chelsa_gfdl_esm4.csv"

# Delete the existing processed CSV
if os.path.exists(output_file):
    os.remove(output_file)
    print(f"🗑️ Deleted {output_file}. Reprocessing will start from scratch.")
else:
    print("✅ No existing CSV found. Proceeding with fresh processing.")

✅ No existing CSV found. Proceeding with fresh processing.


In [ ]:
import rasterio
import pandas as pd
import glob
import os
import re
from rasterio.transform import xy

# Define latitude & longitude range for North America
LAT_MIN, LAT_MAX = 14, 83   # Covers Mexico to Canada
LON_MIN, LON_MAX = -179, 179  # Covers all bird migration longitudes

# Paths
data_folder = "/content/drive/My Drive/CHELSA_Data/"  # Data source in Google Drive
output_file = "/content/chelsa_gfdl_esm4_filtered.parquet"  # ✅ Save locally
batch_size = 100000  # ✅ Save after every 100,000 rows
batch_data = []  # Store rows in memory before writing

# **Check existing Parquet file to find processed files**
processed_files = set()
if os.path.exists(output_file):
    try:
        print("🔎 Loading previously processed files...")
        existing_df = pd.read_parquet(output_file, columns=["Variable", "TimePeriod", "Model", "SSP", "Month"])
        processed_files = set(existing_df.apply(lambda row: f"CHELSA_{row['Model']}_{row['SSP']}_{row['Variable']}_{row['Month']}_{row['TimePeriod']}.tif", axis=1))
        print(f"📌 Found {len(processed_files)} already processed files. Skipping them.")
    except Exception as e:
        print(f"⚠️ Error loading Parquet file: {e}. Continuing with all files.")

# **Get all .tif files for GFDL-ESM4 model (ONLY those with a month in the filename)**
tif_files = [f for f in glob.glob(os.path.join(data_folder, "*.tif")) if "gfdl-esm4" in f]
tif_files = [f for f in tif_files if re.search(r"_(\d{2})_\d{4}_\d{4}_norm\.tif$", os.path.basename(f))]

# **Remove already processed files from processing list**
tif_files = [f for f in tif_files if os.path.basename(f) not in processed_files]
print(f"🔍 {len(tif_files)} new GFDL-ESM4 files to process.")

# **Function to extract scale and offset from a GeoTIFF file**
def extract_metadata(file_path):
    """Extracts scale and offset from a CHELSA GeoTIFF file."""
    with rasterio.open(file_path) as src:
        scale = src.scales[0] if src.scales else 1  # Extract scale
        offset = src.offsets[0] if src.offsets else 0  # Extract offset
        return scale, offset

# **Function to save batch data to Parquet locally**
def save_to_parquet():
    global batch_data  # Access global batch_data

    print(f"🔎 Checking batch size before saving: {len(batch_data)} rows in memory.")

    if batch_data:
        df = pd.DataFrame(batch_data, columns=["Model", "Ensemble", "BiasCorrection", "SSP", "Variable", "Month", "TimePeriod", "Latitude", "Longitude", "Value"])

        print(f"💾 Attempting to write {len(df)} rows to Parquet locally...")

        # ✅ Append new data to existing Parquet file if it exists
        if os.path.exists(output_file):
            existing_df = pd.read_parquet(output_file)
            df = pd.concat([existing_df, df], ignore_index=True)

        df.to_parquet(output_file, engine="pyarrow", index=False)
        print(f"✅ Successfully saved {len(df)} rows to {output_file}")

        batch_data.clear()  # Clear batch to free memory

# **Function to process a single file efficiently**
def process_file(file, file_num, total_files):
    model, ensemble, bias_correction, ssp, variable, month, time_period = parse_filename(file)

    if month is None or month == "":
        print(f"⚠️ Skipping file without a valid month: {file}")
        return

    scale, offset = extract_metadata(file)  # Get scale & offset from metadata
    extracted_data = []

    print(f"🔄 Processing file {file_num}/{total_files}: {os.path.basename(file)} with Scale: {scale}, Offset: {offset}")

    with rasterio.open(file) as src:
        band1 = src.read(1, masked=True)
        transform = src.transform

        rows, cols = band1.shape
        sample_rate = 50

        for row in range(0, rows, sample_rate):
            for col in range(0, cols, sample_rate):
                lon, lat = xy(transform, row, col)
                value = band1[row, col]

                if LAT_MIN <= lat <= LAT_MAX and LON_MIN <= lon <= LON_MAX and value is not None and not pd.isna(value):
                    real_value = (value * scale) + offset  # Apply conversion
                    extracted_data.append([model, ensemble, bias_correction, ssp, variable, month, time_period, lat, lon, real_value])

    print(f"✅ Extracted {len(extracted_data)} rows from {os.path.basename(file)}")

    if extracted_data:
        batch_data.extend(extracted_data)  # Store data in memory

    # **Force a Save Every 100,000 Rows**
    if len(batch_data) >= batch_size:
        save_to_parquet()

# **Process remaining files**
total_files = len(tif_files)
for i, file in enumerate(tif_files):
    process_file(file, i + 1, total_files)  # ✅ Show progress: "Processing file X/Y"

    # **Save Every 10 Files**
    if (i + 1) % 10 == 0:
        print(f"🔄 Processed {i+1}/{total_files} files... Force saving.")
        save_to_parquet()

# **Final Save for Remaining Data**
print("🔎 Checking if any remaining data needs to be saved...")
save_to_parquet()  # Ensure all remaining data is written

print("🎉 GFDL-ESM4 data extraction complete! Saved to:", output_file)

🔍 432 new GFDL-ESM4 files to process.
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_01_2011_2040_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp585, Variable: tasmin, Month: 01, Time Period: 2011-2040
--------------------------------------------------
🔄 Processing file 1/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_01_2011_2040_norm.tif with Scale: 0.1, Offset: -273.15
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_01_2011_2040_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally...
✅ Successfully saved 141735 rows to /content/chelsa_gfdl_esm4_filtered.parquet
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_tasmin_02_2011_2040_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp585, Variable: tasmin, Month: 02, Time Period: 2011-2040
--------------------------------------------------
🔄 Processing file 2/432: CHELSA_gf

🔄 Processing file 146/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_05_2041_2070_norm.tif with Scale: 0.1, Offset: 0.0


✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_05_2041_2070_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally...
✅ Successfully saved 20693310 rows to /content/chelsa_gfdl_esm4_filtered.parquet
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_03_2041_2070_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp370, Variable: tas, Month: 03, Time Period: 2041-2070
--------------------------------------------------
🔄 Processing file 147/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_03_2041_2070_norm.tif with Scale: 0.1, Offset: -273.15
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp370_tas_03_2041_2070_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally...
✅ Successfully saved 20835045 rows to /content/chelsa_gfdl_esm4_filtered.parquet
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp1

🔄 Processing file 150/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_06_2041_2070_norm.tif with Scale: 0.1, Offset: 0.0
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_06_2041_2070_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally...
✅ Successfully saved 21260250 rows to /content/chelsa_gfdl_esm4_filtered.parquet
🔄 Processed 150/432 files... Force saving.
🔎 Checking batch size before saving: 0 rows in memory.
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_05_2041_2070_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp126, Variable: tasmax, Month: 05, Time Period: 2041-2070
--------------------------------------------------
🔄 Processing file 151/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_05_2041_2070_norm.tif with Scale: 0.1, Offset: -273.15
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_05_2041_2070_norm.tif
🔎 Checking batc

🔄 Processing file 153/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_07_2041_2070_norm.tif with Scale: 0.1, Offset: 0.0
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_07_2041_2070_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally...
✅ Successfully saved 21685455 rows to /content/chelsa_gfdl_esm4_filtered.parquet
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_06_2041_2070_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp126, Variable: tasmax, Month: 06, Time Period: 2041-2070
--------------------------------------------------
🔄 Processing file 154/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_06_2041_2070_norm.tif with Scale: 0.1, Offset: -273.15
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_06_2041_2070_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally.

🔄 Processing file 156/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_08_2041_2070_norm.tif with Scale: 0.1, Offset: 0.0
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp585_pr_08_2041_2070_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally...
✅ Successfully saved 22110660 rows to /content/chelsa_gfdl_esm4_filtered.parquet
✅ Filename: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_07_2041_2070_norm.tif
Model: gfdl-esm4, Ensemble: r1i1p1f1, Bias Correction: w5e5, SSP: ssp126, Variable: tasmax, Month: 07, Time Period: 2041-2070
--------------------------------------------------
🔄 Processing file 157/432: CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_07_2041_2070_norm.tif with Scale: 0.1, Offset: -273.15
✅ Extracted 141735 rows from CHELSA_gfdl-esm4_r1i1p1f1_w5e5_ssp126_tasmax_07_2041_2070_norm.tif
🔎 Checking batch size before saving: 141735 rows in memory.
💾 Attempting to write 141735 rows to Parquet locally.

KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("/content/chelsa_gfdl_esm4_filtered.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = pd.read_parquet("/content/chelsa_gfdl_esm4_filtered.parquet")

In [ ]:
#sampling where variable is tas
df.sample(10)

,Model,Ensemble,BiasCorrection,SSP,Variable,Month,TimePeriod,Latitude,Longitude,Value
29998090,gfdl-esm4,r1i1p1f1,w5e5,ssp585,tas,02,2011-2040,38.162361,-140.412639,2857
5582693,gfdl-esm4,r1i1p1f1,w5e5,ssp585,tas,03,2071-2100,56.079027,-157.079306,2791
47018541,gfdl-esm4,r1i1p1f1,w5e5,ssp370,pr,04,2041-2070,32.329027,-46.662639,873
55858699,gfdl-esm4,r1i1p1f1,w5e5,ssp126,tas,06,2071-2100,75.662360,32.087360,2760
21741121,gfdl-esm4,r1i1p1f1,w5e5,ssp126,tasmax,06,2041-2070,56.079027,108.754027,2865
60804159,gfdl-esm4,r1i1p1f1,w5e5,ssp126,pr,10,2041-2070,14.412361,114.170693,2627
41579152,gfdl-esm4,r1i1p1f1,w5e5,ssp370,pr,02,2011-2040,58.162361,-130.412639,271
7418409,gfdl-esm4,r1i1p1f1,w5e5,ssp370,tasmax,08,2071-2100,59.412361,-143.329306,2896
61094943,gfdl-esm4,r1i1p1f1,w5e5,ssp126,pr,12,2041-2070,79.412360,-59.579306,75
41583443,gfdl-esm4,r1i1p1f1,w5e5,ssp370,pr,02,2011-2040,56.079027,-132.079306,2132


In [ ]:
#count by Variable
df.groupby('Variable').count()

,Model,Ensemble,BiasCorrection,SSP,Month,TimePeriod,Latitude,Longitude,Value
Variable,,,,,,,,,
pr,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380
tas,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380
tasmax,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380
tasmin,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380,15307380


In [ ]:
import pandas as pd

future_climate = pd.read_parquet("/content/chelsa_gfdl_esm4_filtered.parquet")

In [ ]:
# Remove rows where the TimePeriod is '2071-2100'
future_climate = future_climate[future_climate['TimePeriod'] != '2071-2100']

# Drop the Ensemble/BiasCorrection columns since it has only one unique value
future_climate = future_climate.drop(columns=['Model, Ensemble, BiasCorrection'])

In [ ]:
#count records by TimePeriod
future_climate.groupby('TimePeriod').count()

,SSP,Variable,Month,Latitude,Longitude,Value
TimePeriod,,,,,,
2011-2040,20409840,20409840,20409840,20409840,20409840,20409840
2041-2070,20409840,20409840,20409840,20409840,20409840,20409840


In [ ]:
future_climate.groupby('Variable').count()

,SSP,Month,TimePeriod,Latitude,Longitude,Value
Variable,,,,,,
pr,10204920,10204920,10204920,10204920,10204920,10204920
tas,10204920,10204920,10204920,10204920,10204920,10204920
tasmax,10204920,10204920,10204920,10204920,10204920,10204920
tasmin,10204920,10204920,10204920,10204920,10204920,10204920


In [ ]:
future_climate.sample(10)

,SSP,Variable,Month,TimePeriod,Latitude,Longitude,Value
44868546,ssp126,tas,01,2041-2070,43.995694,-12.495973,2858
53049378,ssp370,tas,01,2011-2040,63.162360,-130.829306,2524
30841188,ssp585,tas,05,2011-2040,41.912361,34.170694,2883
30688032,ssp370,tasmax,05,2011-2040,47.329027,-71.662639,2880
1393367,ssp585,tasmin,10,2011-2040,25.662361,-149.995972,2969
28529338,ssp126,tasmin,09,2011-2040,63.162360,-82.912639,2758
57685937,ssp585,pr,04,2041-2070,14.412361,92.504027,315
35441698,ssp126,tasmin,02,2041-2070,78.995694,-88.329306,2418
38091689,ssp370,tasmax,08,2041-2070,31.079027,-98.329306,3106
10071556,ssp585,tasmax,04,2011-2040,78.995694,87.920693,2592


In [ ]:
#save the final file
future_climate.to_parquet("/content/future_climate_final.parquet")

In [ ]:
from google.colab import files
files.download("/content/future_climate_final.parquet")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>